In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import altair as alt
alt.renderers.enable('notebook')
from vega_datasets import data

from math import pi

# code reference at blog.prototypr.io/interactive-maps-with-python-part-1-aa1563dbe5a9

In [184]:

folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

# type of graph in 'json' format
json1_file = open('stackedBar.json')
json1_str = json1_file.read()
folium.CircleMarker(location=([40.738, -73.98]),
                            radius=10,
                            color="#0A8A9F",
                            popup=folium.Popup(max_width=550).add_child(
        folium.VegaLite(json1_str, width=550, height=250)),
                            fill=True).add_to(folium_map)
folium_map
#type(json1_str)

In [185]:
# load csv of all station information
subway_data = pd.read_csv("Stations.csv")

# need to get lat/long of all stations to draw circle
# need to figure out how to make interactable (default pics)
subway_data

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,517,517,S15,SIR,Staten Island,Prince's Bay,SI,SIR,Open Cut,40.525507,-74.200064,St George,Tottenville
492,518,518,S14,SIR,Staten Island,Pleasant Plains,SI,SIR,Embankment,40.522410,-74.217847,St George,Tottenville
493,519,519,S13,SIR,Staten Island,Richmond Valley,SI,SIR,Open Cut,40.519631,-74.229141,St George,Tottenville
494,522,522,S09,SIR,Staten Island,Tottenville,SI,SIR,At Grade,40.512764,-74.251961,St George,NaN


In [186]:
locations = subway_data.groupby("GTFS Latitude").first()
locations = locations.loc[:, ["GTFS Longitude",
                              "Stop Name",
                              "Borough"]]
locations.head()

,GTFS Longitude,Stop Name,Borough
GTFS Latitude,,,
40.512764,-74.251961,Tottenville,SI
40.516578,-74.242096,Arthur Kill,SI
40.519631,-74.229141,Richmond Valley,SI
40.522410,-74.217847,Pleasant Plains,SI
40.525507,-74.200064,Prince's Bay,SI


In [187]:
def pie_chart_maker(randInt):
    # Pie chart, where the slices will be ordered and plotted counter-clockwise:
    labels = 'one','two'
    #colors = 'orange','gray'
    sizes = [randInt,(1-randInt)]
    explode = (0,0)

    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, explode=explode,
            shadow=True, startangle=90)
    ax1.legend(labels, loc='best',
               fontsize=10)

    #ax1.rcParams["figure.figsize"] = [4,3]

    #ax1.pie(sizes, labels,autopct='%1.1f%%')
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    
    return ax1

    #plt.show()

In [188]:
def make_pie():    
    output_file("pie.html")

    x = {
        'United States': 157,
        'United Kingdom': 93,
        'Japan': 89,
        'China': 63,
        'Germany': 44,
        'India': 42,
        'Italy': 40,
        'Australia': 35,
        'Brazil': 32,
        'France': 31,
        'Taiwan': 31,
        'Spain': 29
    }

    data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    data['color'] = Category20c[len(x)]

    p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
               tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend_field='country', source=data)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    
    return show(p)
    #show(p)

In [189]:
chart = alt.Chart(data.cars.url).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N'
)

chart.save('chart.json')

In [202]:
# plot circles at each station
def plot_stations(station):
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=10,
                            tiles="CartoDB dark_matter",
                            width='100%')
    
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, row["GTFS Longitude"] = longitude
        
        # number of spotted officers plotted
        
        # generate popup message shown on click
        popup_text = "stop name: {}"
        popup_text = popup_text.format(row["Stop Name"])
        #popup_text = json1_str
        #popup_text = folium.Popup(max_width=550).add_child(
                                  #folium.VegaLite(json1_str, width=550, height=250))
        
        # radius of circles
        radius = 3
        # color of circles (can change conditionally, by borough)
        if(row["Borough"] == 'Bx'):
            #Bronx = red
            color='#FF0000'
        elif(row["Borough"] == 'Bk'):
            #Brooklyn=yellow
            color='#ffff00'
        elif(row["Borough"] == 'M'):
            #Manhattan=green
            color='#00ff00'
        elif(row["Borough"] == 'Q'):
            #Queens=orange
            color='#ffa500'
        elif(row["Borough"] == 'SI'):
            #Staten Island=Purple
            color="#0A8A9F"
        else:
            #Missing Borough colored blue
            color='#800080'
        #print(color)
        folium.CircleMarker(location=(index,
                                      row["GTFS Longitude"]),
                            radius=np.random.randint(low=1,high=10),
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
    return folium_map

In [203]:
# call function plot_stations of each subway stop
plot_stations(locations)

In [192]:
folium_map = plot_stations(locations)
#folium_map

In [193]:
folium_map.save("ClickableSubwayMapStackedBarColoredBoroughs.html")